# Initialization

In [ ]:
%matplotlib inline

In [ ]:
# data access
import io
import sqlalchemy as sa

# data handling
import json

# internet
import requests

# data analysis
import numpy as np
import pandas as pd

import scipy
from scipy import stats
import statsmodels.api as sm
#import scikit-learn as sk

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# system packages
from imp import reload

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.style.use('seaborn')

In [ ]:
# my own libs
from libs.gov_eneff import rt

rt.init('config/config.json')
logger = rt.logger

In [ ]:
LANG='en'

In [ ]:
from libs.gov_eneff import db
reload(db)

# Load Data

In [ ]:
db = db.DB()
db.load(source='local')

In [ ]:
md_cols     = db.md_cols.copy()

df_org      = db.df_org.copy()
df_declr    = db.df_declr.copy()
df_bld      = db.df_bld.copy()

## Metadata

In [ ]:
declrs_f_seq = pd.Index(pd.read_csv('data/declrs_fields_sequence.txt', header=None)[0])
print(len(declrs_f_seq))

In [ ]:
declr_buildings_f_seq = pd.Index(pd.read_csv('data/declr_buildings_fields_sequence.txt', header=None)[0])
(print(len(declr_buildings_f_seq)))

# Missing Data

## Missing Data - Org Level

In [ ]:
# taking data from the database instead of db.df_declr as the former may not contain all properties

stmt = sa.sql.text('''
select *
  from declrs
 where fake=0
''')
df_declr_tmp= pd.read_sql_query(stmt, con=rt.db_engine['energy'])
print(df_declr_tmp.shape)

In [ ]:
df_declr_misses = df_declr_tmp.query('year < 2019')\
    .melt(id_vars='year').groupby(['year', 'variable'])\
    .apply(lambda x: round(100*x['value'].isnull().sum()/len(x['value']), 2))\
    .rename('pct')\
    .to_frame().reset_index()\
    .pivot(index='variable', columns='year')

# the more value, the worse
df_declr_misses['trend_relative'] = df_declr_misses.apply(lambda x: x.loc['pct'].diff().sum().round(2), axis=1)

In [ ]:
df_declr_misses.columns = df_declr_misses.columns.droplevel()
df_declr_misses = df_declr_misses.merge(md_cols.query('table_name == "declrs"')[['column_name', 'column_comment']].set_index('column_name'), 
                      left_index=True, right_index=True, how='left').fillna('')

# description
df_declr_misses['hm_name'] = df_declr_misses['column_comment'] + ' (' + df_declr_misses.index + ')'

logger.debug('Shape of df_declr_misses is {}'.format(df_declr_misses.shape))

In [ ]:
declrs_f_seq_complemented = declrs_f_seq.append(df_declr_misses.index[~df_declr_misses.index.isin(declrs_f_seq)])

# this is just to mark all rows that they have existed to this point
df_declr_misses['C'] = '1'

df_declr_misses = df_declr_misses\
    .reindex(declrs_f_seq_complemented)\
    .dropna(subset=['C'])\
    .drop(['C'], axis=1)\
    .drop_duplicates()\
    .copy()

logger.debug('Shape of df_declr_misses after the index reorder is {}'.format(df_declr_misses.shape))

In [ ]:
# fancy graphs

fig, axs = plt.subplots(1, 1, figsize=(14, 16))

ax = axs
p = sns.heatmap(df_declr_misses.loc[:, ('hm_name', 2014, 2015, 2016, 2017, 2018)].set_index('hm_name'), 
                square=False, cmap=plt.get_cmap('OrRd'), ax=ax)

p.set_ylabel('')

fig.tight_layout()
plt.savefig('./images/declrs_misses_heatmap.pdf')
plt.savefig('./images/declrs_misses_heatmap.png', dpi=150)

In [ ]:
# fancy graphs

fig, axs = plt.subplots(2, 1, figsize=(8, 12), gridspec_kw={'height_ratios': [1, 20]})

ax = axs[0]
p = sns.heatmap(df_declr_misses.loc[:, (2014, 2015, 2016, 2017, 2018)].mean().round(1).to_frame().T, 
                square=False, cmap=plt.get_cmap('OrRd'), 
                vmin=0, vmax=100, annot=True, fmt='.1f', annot_kws={"size": 20}, cbar=True, ax=ax)

for t in ax.texts: t.set_text(t.get_text() + " %")
    
ax.set_ylabel('')
ax.set_yticklabels(labels='')
ax.set_xticklabels(ax.get_xticklabels(), fontsize=16)

# remove cbar
# TBC

ax = axs[1]
p = sns.heatmap(df_declr_misses.loc[:, ('hm_name', 2014, 2015, 2016, 2017, 2018)].set_index('hm_name'), 
                square=False, cmap=plt.get_cmap('OrRd'), ax=ax)
ax.set_ylabel('')
ax.set_xticklabels('')
ax.set_yticklabels('')

fig.tight_layout()
plt.savefig('./images/declrs_misses_heatmap_no_labels.png', dpi=150)
plt.savefig('./images/declrs_misses_heatmap_no_labels.pdf')

In [ ]:
dft2 = pd.read_excel('data/local_db/declr_attrs.xlsx', index_col=0)

# completeness
dft2 = dft2.merge(100-df_declr_misses.loc[:, (2014, 2015, 2016, 2017, 2018)].round(1), left_index=True, right_index=True, how='left').fillna('-')

# report in the format of a TeX table
with open('export/90_appendix_A_declr_attributes.tex', 'w', encoding='utf8') as f:
    dft2.to_latex(f, longtable=True)

## Missing Data - Building Level (Declaration)

In [ ]:
stmt = sa.sql.text('''
select d.year,
       bld.*,
       -- cold water
       cw.id cold_water_id, cw.cold_water_type, cw.cold_water_consumption, cw.cold_water_unit, cw.cold_water_payment,
       -- hot water
       hw.id hot_water_id,  hw.hot_water_type,  hw.hot_water_consumption,  hw.hot_water_unit,  hw.hot_water_payment,
       hwj.name hot_water_join_name,
       -- cold water
       ee.id electric_id,   ee.electric_type,   ee.electric_consumption,   ee.electric_unit,   ee.electric_payment,
       -- heat
       ht.id heat_id,       ht.heat_type,       ht.heat_consumption,       ht.heat_unit,       ht.heat_payment,
       -- OKVED
       voc_okved.code main_okved_code, voc_okved.label main_okved_label,
       voc_efficiency_class.label efficiency_class_name
  from declr_buildings bld 
       inner join declrs d on bld.id_declr = d.id
       inner join orgs o on d.id_org = o.id left join voc_okved on o.id_voc_okved = voc_okved.id
       left join voc_building_types   on bld.id_voc_building_type = voc_building_types.id
       left join voc_efficiency_class on bld.id_voc_efficiency_class = voc_efficiency_class.id
       -- adding the cold water consumption
       left join (select *, 'cw' energy_type from declr_cold_water where fake = 0) cw on bld.id = cw.id_declr_building -- and bld.id_declr = heat.id_declr
       -- adding the heat consumption
       left join (select *, 'hw' energy_type from declr_hot_water  where fake = 0) hw on bld.id = hw.id_declr_building -- and bld.id_declr = heat.id_declr
       -- adding the heat consumption
       left join (select * from voc_hot_water_join where fake = 0) hwj on bld.id_voc_hot_water_join = hwj.id
       left join (select *, 'ee' energy_type from declr_electric   where fake = 0) ee on bld.id = ee.id_declr_building -- and bld.id_declr = heat.id_declr
       -- adding the heat consumption
       left join (select *, 'ht' energy_type from declr_heat       where fake = 0) ht on bld.id = ht.id_declr_building -- and bld.id_declr = heat.id_declr
 where d.fake    = 0
   and bld.fake  = 0
   and o.fake    = 0
   and voc_okved.fake = 0
   and voc_efficiency_class.fake = 0
 order by bld.id_declr, bld.id
''')

df_bld_tmp= pd.read_sql_query(stmt, con=rt.db_engine['energy'])
print(df_bld_tmp.shape)

In [ ]:
df_bld_misses = df_bld_tmp.query('year in (2014, 2015, 2016, 2017, 2018)')\
    .melt(id_vars='year').groupby(['year', 'variable'])\
    .apply(lambda x: round(100*x['value'].isnull().sum()/len(x['value']), 2))\
    .rename('pct')\
    .to_frame().reset_index()\
    .pivot(index='variable', columns='year')

# the more value, the worse
df_bld_misses['trend_relative'] = df_bld_misses.apply(lambda x: x.loc['pct'].diff().sum().round(2), axis=1)

In [ ]:
df_bld_misses.columns = df_bld_misses.columns.droplevel()
df_bld_misses = df_bld_misses.merge(md_cols.query('table_name == "declr_buildings"')[['column_name', 'column_comment']].set_index('column_name'), 
                      left_index=True, right_index=True, how='left').fillna('')

# description
df_bld_misses['hm_name'] = df_bld_misses['column_comment'] + ' (' + df_bld_misses.index + ')'

logger.debug('Shape of df_bld_misses is {}'.format(df_bld_misses.shape))

In [ ]:
declr_buildings_f_seq_complemented = \
    declr_buildings_f_seq.append(df_bld_misses.index[~df_bld_misses.index.isin(declr_buildings_f_seq)])

# this is just to mark all rows that they have existed to this point
df_bld_misses['C'] = '1'

df_bld_misses = df_bld_misses\
    .reindex(declr_buildings_f_seq_complemented)\
    .dropna(subset=['C'])\
    .drop(['C'], axis=1)\
    .drop_duplicates()\
    .copy()

logger.debug('Shape of df_declr_misses after the index reorder is {}'.format(df_bld_misses.shape))

In [ ]:
# fancy graphs

fig, axs = plt.subplots(1, 1, figsize=(15, 45))

ax = axs
p = sns.heatmap(df_bld_misses.loc[:, ('hm_name', 2014, 2015, 2016, 2017, 2018)].set_index('hm_name'), 
                square=False, cmap=plt.get_cmap('OrRd'), ax=ax)

p.set_ylabel('')

fig.tight_layout()
plt.savefig('./images/bld_misses_heatmap.pdf')
plt.savefig('./images/bld_misses_heatmap.png', dpi=150)

In [ ]:
# fancy graphs

fig, axs = plt.subplots(2, 1, figsize=(8, 12), gridspec_kw={'height_ratios': [1, 20]})

ax = axs[0]
p = sns.heatmap(df_bld_misses.loc[:, (2014, 2015, 2016, 2017, 2018)].mean().round(1).to_frame().T, 
                square=False, cmap=plt.get_cmap('OrRd'), 
                vmin=0, vmax=100, annot=True, fmt='.1f', annot_kws={"size": 20}, cbar=True, ax=ax)

for t in ax.texts: t.set_text(t.get_text() + " %")
    
ax.set_ylabel('')
ax.set_yticklabels(labels='')
ax.set_xticklabels(ax.get_xticklabels(), fontsize=16)

# remove cbar
# TBC

ax = axs[1]
p = sns.heatmap(df_bld_misses.loc[:, ('hm_name', 2014, 2015, 2016, 2017, 2018)].set_index('hm_name'), 
                square=False, cmap=plt.get_cmap('OrRd'), ax=ax)

ax.set_ylabel('')
ax.set_xticklabels('')
ax.set_yticklabels('')

fig.tight_layout()
plt.savefig('./images/bld_misses_heatmap_no_labels.png', dpi=150)
plt.savefig('./images/bld_misses_heatmap_no_labels.pdf')

In [ ]:
dft2 = pd.read_excel('data/local_db/bld_attrs.xlsx', index_col=0)

# completeness
dft2 = dft2.merge(100-df_bld_misses.loc[:, (2014, 2015, 2016, 2017, 2018)].round(1), left_index=True, right_index=True, how='left').fillna('-')

# report in the format of a TeX table
with open('export/90_appendix_A_bld_attributes.tex', 'w', encoding='utf8') as f:
    dft2.to_latex(f, longtable=True)

## Consumption

### Cold Water

In [ ]:
dft = df_bld_tmp[['year', 'cold_water_consumption']]\
    .groupby('year')['cold_water_consumption'].apply(lambda x: x.isna().mean()).to_frame().T

dft = dft.round(4)*100

# fancy graphs

fig, axs = plt.subplots(1, 1, figsize=(4, 1))

ax = axs
sns.heatmap(dft, square=False, cmap=plt.get_cmap('OrRd'), 
            vmin=0, vmax=100, annot=True, fmt='.1f', annot_kws={"size": 12}, cbar=False, ax=ax)

for t in ax.texts: t.set_text(t.get_text() + " %")

ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticklabels(labels='')
ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)

fig.tight_layout()
plt.savefig('./images/cw_misses.png', dpi=150)
plt.savefig('./images/cw_misses.pdf')

### Hot Water

In [ ]:
dft = df_bld_tmp.query('hot_water_join_name != "none" and hot_water_join_name == hot_water_join_name')[['year', 'hot_water_consumption']]\
    .groupby('year')['hot_water_consumption'].apply(lambda x: x.isna().mean()).to_frame().T

dft = dft.round(4)*100

# fancy graphs

fig, axs = plt.subplots(1, 1, figsize=(4, 1))

ax = axs
sns.heatmap(dft, square=False, cmap=plt.get_cmap('OrRd'), 
            vmin=0, vmax=100, annot=True, fmt='.1f', annot_kws={"size": 12}, cbar=False, ax=ax)

for t in ax.texts: t.set_text(t.get_text() + " %")

ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticklabels(labels='')
ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)

fig.tight_layout()
plt.savefig('./images/hw_misses.png', dpi=150)
plt.savefig('./images/hw_misses.pdf')

### EE

In [ ]:
dft = df_bld_tmp[['year', 'electric_consumption']]\
    .groupby('year')['electric_consumption'].apply(lambda x: x.isna().mean()).to_frame().T

dft = dft.round(4)*100

# fancy graphs

fig, axs = plt.subplots(1, 1, figsize=(4, 1))

ax = axs
sns.heatmap(dft, square=False, cmap=plt.get_cmap('OrRd'), 
            vmin=0, vmax=100, annot=True, fmt='.1f', annot_kws={"size": 12}, cbar=False, ax=ax)

for t in ax.texts: t.set_text(t.get_text() + " %")

ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticklabels(labels='')
ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)

fig.tight_layout()
plt.savefig('./images/ee_misses.png', dpi=150)
plt.savefig('./images/ee_misses.pdf')

### Heat

In [ ]:
dft = df_bld_tmp.query('heated_area > 0')[['year', 'heat_consumption']]\
    .groupby('year')['heat_consumption'].apply(lambda x: x.isna().mean()).to_frame().T

dft = dft.round(4)*100

# fancy graphs

fig, axs = plt.subplots(1, 1, figsize=(4, 1))

ax = axs
sns.heatmap(dft, square=False, cmap=plt.get_cmap('OrRd'), 
            vmin=0, vmax=100, annot=True, fmt='.1f', annot_kws={"size": 12}, cbar=False, ax=ax)

for t in ax.texts: t.set_text(t.get_text() + " %")

ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticklabels(labels='')
ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)

fig.tight_layout()
plt.savefig('./images/ht_misses.png', dpi=150)
plt.savefig('./images/ht_misses.pdf')

## Export

In [ ]:
import string

sheets = {'Декларации - Общее':  {'data': df_declr_misses},
          'Декларации - Здания': {'data': df_bld_misses}}

file_name = './export/%s' % 'missing_data_stats.xlsx'

xls_writer = pd.ExcelWriter(file_name,
                            engine='xlsxwriter',
                            options={'strings_to_urls': False})  # 'strings_to_numbers': True

logger.info('Saving to %s' % file_name)

for key, val in sheets.items():

    # Convert the dataframe to an XlsxWriter Excel object.
    sheet_name = key

    val['data'].reset_index().to_excel(xls_writer, index=False, sheet_name=sheet_name)

    # Set the column width and format.
    workbook  = xls_writer.book
    worksheet = xls_writer.sheets[sheet_name]
    sheet_data_range = (string.ascii_uppercase[0] + '1', 
                        string.ascii_uppercase[val['data'].shape[1]] + str(val['data'].shape[0] + 1))
    worksheet.set_column('A:A', width = 40,  cell_format = workbook.add_format().set_align('left'))
    worksheet.set_column('B:F', width = 10,  cell_format = workbook.add_format().set_align('center'))
    worksheet.set_column('G:G', width = 100, cell_format = workbook.add_format().set_align('left'))
    
    # Set the autofilter.
    worksheet.autofilter('%s:%s' % sheet_data_range)

xls_writer.save()

# ===== Objects

## Basic Stats

In [ ]:
# xx-small, x-small, small, medium, large, x-large, xx-large, larger, smaller

SMALL_SIZE = 10
MEDIUM_SIZE = 12
LARGE_SIZE = 14

plt.rc('font',   size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes',   titlesize=LARGE_SIZE)      # fontsize of the axes title
plt.rc('axes',   labelsize=LARGE_SIZE)      # fontsize of the x and y labels
plt.rc('xtick',  labelsize=LARGE_SIZE)      # fontsize of the tick labels
plt.rc('ytick',  labelsize=LARGE_SIZE)      # fontsize of the tick labels
plt.rc('legend', fontsize=LARGE_SIZE)       # legend fontsize
plt.rc('figure', titlesize=LARGE_SIZE)      # fontsize of the figure title

In [ ]:
df_declr['year'].value_counts().mean()

In [ ]:
df_declr['year'].value_counts().sort_index()

In [ ]:
df_declr[df_declr['employees'] == 0].shape

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

df_declr.groupby('year')['year'].count().plot(kind='bar', ax=ax)
df_declr.groupby('year')['id_org'].count().plot(kind='bar', ax=ax)

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Количество деклараций')
elif LANG == 'en':
    ax.set_xlabel('Year')
    ax.set_ylabel('Number of declarations')
    
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)

fig.tight_layout()
fig.savefig('images/declrs_by_years.png', dpi=300)
fig.savefig('images/declrs_by_years.pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_declr.groupby('year')['total_consumption_tut'].sum()/1000000
dft.plot(kind='bar', ax=ax)

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Потребление топлива, млн. т.у.т.')
elif LANG == 'en':
    ax.set_xlabel('Year')
    ax.set_ylabel('Energy consumption, mln. t.u.t.')
    
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
ax.yaxis.set_major_formatter(plt.FormatStrFormatter('%.1f'))

fig.tight_layout()
fig.savefig('images/declrs_tut_by_years.png', dpi=300)
fig.savefig('images/declrs_tut_by_years.pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

df_bld.groupby('year')['year'].count().plot(kind='bar', ax=ax)

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Количество зданий')
elif LANG == 'en':    
    ax.set_xlabel('Year')
    ax.set_ylabel('Number of buildings')
    
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)

fig.tight_layout()
fig.savefig('images/bld_by_years.png', dpi=300)
fig.savefig('images/bld_by_years.pdf')

In [ ]:
df_bld[df_bld['employees'] == 0].shape

In [ ]:
# df_bld[df_bld['guests'] == 0].shape[0]/len(df_bld)

## Floor Area

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld.groupby('year')['floor_area'].sum()/10000000

dft.plot(kind='bar', ax=ax)
if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Общая площадь, млн. кв. м')
elif LANG == 'en':     
    ax.set_xlabel('Year')
    ax.set_ylabel('Floor area, mln. of square meters')
    
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig('images/bld_floor_area_years.png', dpi=300)
fig.savefig('images/bld_floor_area_years.pdf')

In [ ]:
df_bld.groupby('year')['floor_area'].mean()

Checking for any buildings with unusually high heated_area

In [ ]:
df_bld.sort_values('floor_area', ascending=False).head(5)[['id_org', 'floor_area']]\
    .merge(df_org[['label']], left_on='id_org', right_index=True)

In [ ]:
from libs.gov_eneff import preprocessing
bld_filter_set = preprocessing.BldFilter()

In [ ]:
dft = bld_filter_set.f1(df_bld) #.groupby('year')['heated_area'].sum()[['year', 'heated_area']]
dft.groupby('year')['floor_area'].mean().round(1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld.groupby('year')['heated_area'].sum()/10000000

dft.plot(kind='bar', ax=ax)
if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Отапливаемая площадь, млн. кв. м')
elif LANG == 'en':
    ax.set_xlabel('Year')
    ax.set_ylabel('Heated area, mln. squared meters')
    
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig('images/bld_heated_area_years.png', dpi=300)
fig.savefig('images/bld_heated_area_years.pdf')

In [ ]:
df_bld.groupby('year')['heated_area'].mean().round(1)

In [ ]:
df_bld.groupby('year')['heated_area'].median().round(1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

#dft = df_bld.query('heated_area <= 100000')
dft = df_bld.query('heated_area > 0 and heated_area == heated_area').copy()
dft['heated_area_log10'] = np.log10(dft['heated_area'])

#sns.catplot(x='year', y='heated_area_log10', data=dft, kind='violin') #, width=0.6)
sns.boxplot(x='year', y='heated_area_log10', data=dft, width=0.6, color='#33CC66')

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Отапливаемая площадь, млн. кв. м')
elif LANG == 'en':
    ax.set_xlabel('Year')
    ax.set_ylabel('$log_{10}{heated\_area}$')
    
#plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)

ax.set_yticks(range(8))
#ax.set_yticklabels([str(n) for n in range(7)])

fig.tight_layout()
fig.savefig('images/bld_heated_area_years_boxplot.png', dpi=300)
fig.savefig('images/bld_heated_area_years_boxplot.pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

#dft = df_bld.query('heated_area <= 100000')
dft = df_bld.query('heated_area > 0 and heated_area == heated_area').copy()
dft['heated_area_log10'] = np.log10(dft['heated_area'])

#sns.catplot(x='year', y='heated_area_log10', data=dft, kind='violin') #, width=0.6)
sns.boxplot(x='year', y='heated_area_log10', data=dft, width=0.6, color='#33CC66')

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Отапливаемая площадь, млн. кв. м')
elif LANG == 'en':
    ax.set_xlabel('Year')
    ax.set_ylabel('$log_{10}{heated\_area}$')
    
#plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)

ax.set_yticks(range(8))
#ax.set_yticklabels([str(n) for n in range(7)])

fig.tight_layout()
fig.savefig('images/bld_heated_area_years_boxplot.png', dpi=300)
fig.savefig('images/bld_heated_area_years_boxplot.pdf')

#  Geo Distribution

- Geomapping, GDAL for Windows: https://sandbox.idre.ucla.edu/sandbox/tutorials/installing-gdal-for-windows

Maps: 
- https://data.nextgis.com/ru/
- http://gisgeo.org/data.html
- http://download.geofabrik.de/russia.html
- https://habr.com/ru/post/321710/
- https://wiki.openstreetmap.org/wiki/Shapefiles

## On Scatterplot

In [ ]:
dft.shape

In [ ]:
print(9/6)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 7))

dft = df_bld[['dd_geo_lat', 'dd_geo_lon', 'dd_okved_code_l1']].drop_duplicates()
print(dft.shape)
print((dft['dd_geo_lat'].max()-dft['dd_geo_lat'].min())/(dft['dd_geo_lon'].max()-dft['dd_geo_lon'].min()))
print(dft['dd_geo_lat'].isna().mean())

# jittering
jitter_size = 0.05
dft['dd_geo_lat'] = dft['dd_geo_lat'] + np.random.uniform(low=-jitter_size, high=+jitter_size, size=len(dft))
dft['dd_geo_lon'] = dft['dd_geo_lon'] + np.random.uniform(low=-jitter_size, high=+jitter_size, size=len(dft))

sns.scatterplot(data=dft, x='dd_geo_lon', y='dd_geo_lat', hue='dd_okved_code_l1', marker='+',
                alpha=0.7, ax=ax) #, palette=diverging_colors)

if LANG =='ru':
    ax.set_xlabel('Долгота')
    ax.set_ylabel('Широта')
elif LANG == 'en':
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
ax.legend([])

from matplotlib.ticker import FormatStrFormatter
ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))

fig.tight_layout()
fig.savefig('images/bld_geo_naive.png', dpi=300)
fig.savefig('images/bld_geo_naive.pdf')

## On Map

In [ ]:
# load libraries
import geopandas as gpd

In [ ]:
# set the filepath and load in a shapefile
fp = "maps/Moscow_region.shp"

map_df = gpd.read_file(fp, encoding='utf-8')

In [ ]:
# check data type so we can see that this is not a normal dataframe, but a GEOdataframe
map_df.head(2)

In [ ]:
map_df.geometry.bounds['minx'].min()*0.9

In [ ]:
df_bld['ya_geo_lon'] = df_bld['ya_geo_pos'].apply(lambda x: float(x.split(' ')[0]) if pd.notna(x) else np.nan)
df_bld['ya_geo_lat'] = df_bld['ya_geo_pos'].apply(lambda x: float(x.split(' ')[1]) if pd.notna(x) else np.nan)

In [ ]:
# now let's preview what our map looks like with no data in it
#https://github.com/bendoesdata/make-a-map-geopandas/blob/master/Let's%20make%20a%20map!%20Geopandas%20and%20Matplotlib.ipynb

import pyproj

# yandex geocoder provide the geo information on more objects
geocoder = 'yandex'

if geocoder == 'dadata':
    # case 1: DaData
    df_bld['geo_lon'], df_bld['geo_lat'] = df_bld['dd_geo_lon'], df_bld['dd_geo_lat']
elif geocoder == 'yandex':
    # case 2: Yandex
    df_bld['geo_lon'], df_bld['geo_lat'] = df_bld['ya_geo_lon'], df_bld['ya_geo_lat']

dft = df_bld[['geo_lon', 'geo_lat', 'dd_okved_code_l1']].drop_duplicates()
print(dft.shape)
#print((dft['dd_geo_lat'].max()-dft['dd_geo_lat'].min())/(dft['dd_geo_lon'].max()-dft['dd_geo_lon'].min()))
#print(dft['dd_geo_lat'].isna().mean())

# jittering
jitter_size = 0.02
dft['geo_lon'] = dft['geo_lon'] + np.random.uniform(low=-jitter_size, high=+jitter_size, size=len(dft))
# for some reason the -0.2 offset is required to place points to where they seem to belong
dft['geo_lat'] = dft['geo_lat'] + np.random.uniform(low=-jitter_size, high=+jitter_size, size=len(dft)) - 0.2

# https://data.nextgis.com/ru/faq/
# Данные находятся в системе координат - широта/долгота WGS84 (EPSG: 4326). 
# Система координат отображения данных, используемая для проектов Mercator - EPSG: 3857.
inProj  = pyproj.Proj(init='epsg:4326')
outProj = pyproj.Proj(init='epsg:3857')

dft['geo_lon_3857'], dft['geo_lat_3857'] = pyproj.transform(inProj, outProj, 
                                                              dft['geo_lon'].to_numpy(), 
                                                              dft['geo_lat'].to_numpy())

# moving all points which are too far from the map to the bounds
minx = map_df.geometry.bounds['minx'].min()
dft['geo_lon_3857'] = dft['geo_lon_3857'].apply(lambda x: minx if x < minx else x)
maxx = map_df.geometry.bounds['maxx'].max()
dft['geo_lon_3857'] = dft['geo_lon_3857'].apply(lambda x: maxx if x > maxx else x)
miny = map_df.geometry.bounds['miny'].min()
dft['geo_lat_3857'] = dft['geo_lat_3857'].apply(lambda y: miny if y < miny else y)
maxy = map_df.geometry.bounds['maxy'].max()
dft['geo_lat_3857'] = dft['geo_lat_3857'].apply(lambda y: maxy if y > maxy else y)

fig, ax = plt.subplots(1, 1, figsize=(12, 12))

map_df.plot(ax=ax, linewidth=0.8, edgecolor='0.5', color='white')

ax.axis('off')

sns.scatterplot(data=dft, x='geo_lon_3857', y='geo_lat_3857', hue='dd_okved_code_l1', marker='+',
                alpha=0.7, ax=ax) #, palette=diverging_colors)

if LANG =='ru':
    ax.set_xlabel('Долгота')
    ax.set_ylabel('Широта')
elif LANG == 'en':
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
ax.legend([])

fig.savefig(f'images/bld_geo_{geocoder}.png', dpi=300, bbox_inches='tight', pad_inches=0)
fig.savefig(f'images/bld_geo_{geocoder}.pdf', bbox_inches='tight', pad_inches=0)

# Building Age

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(8, 5), gridspec_kw = {'height_ratios':[7, 1]})

dft = df_bld.query('building_age <= 150')['building_age']

ax_dp = axs[0]
sns.distplot(dft, ax=ax_dp, bins=50)

if LANG =='ru':
    ax_dp.set_xlabel('Возраст здания')
    ax_dp.set_ylabel('Плотность распреления')
elif LANG == 'en':
    ax_dp.set_xlabel('Age')
    ax_dp.set_ylabel('Density')
    
ax_bp = axs[1]

sns.boxplot(dft, ax=ax_bp)
ax_bp.set_xlabel('')
ax_bp.set_xlim(ax_dp.get_xlim())

fig.tight_layout()
fig.savefig('images/bld_age.png', dpi=300)
fig.savefig('images/bld_age.pdf')

# ===== Activity

# OKVED

## Reliability of declrs.main_okved - Poor

In [ ]:
# Percentage of organizations changed their main OKVED over the years

Are there organization which have changed their main OKVED over the years?

In [ ]:
# organization which have changed their main OKVED over the years
dft = df_declr[~df_declr['main_okved'].isnull()][['id_org', 'main_okved']].drop_duplicates()\
    .groupby(['id_org'])['id_org', 'main_okved']\
    .filter(lambda x: len(x.drop_duplicates()) > 1)\
    .sort_values('id_org')['id_org'].drop_duplicates()

In [ ]:
print(len(dft))
print(100*len(dft)/len(df_declr[~df_declr['main_okved'].isnull()]['id_org'].drop_duplicates()))

In [ ]:
df_declr_okved_amb = df_declr.query('id_org in @dft')[['id_org', 'year', 'main_okved']]
print(df_declr_okved_amb.shape)
# drop records with the missing OKVED
df_declr_okved_amb = df_declr_okved_amb[df_declr_okved_amb['main_okved'].notna()]
print(df_declr_okved_amb.shape)

In [ ]:
df_declr_okved_amb.pivot(index='id_org', columns='year').sample(10).fillna('')

In [ ]:
# Intersection of OKVED and OKVED2
stmt = sa.sql.text('''
select code
  from voc_okved
 where code != ''
   and fake = 0  
intersect
select code
  from voc_okved2
 where code != ''
   and fake = 0
''')
okved12_intersect = pd.read_sql_query(stmt, con=rt.db_engine['energy'])
print(okved12_intersect.shape)

In [ ]:
# let's check whether we can still get away with matching all missing codes solely to OKVED
# the idea is to check whether for any case with the mixed codes there are no entries of the codes in both OKVED and OKVED2

# there should be zero records if there are no such codes
print(len(df_declr_okved_amb[df_declr_okved_amb['main_okved'].isin(okved12_intersect['code'])]))

# unfortunately, it's not possible to remove ambiguity

# df_declr_okved_amb.merge(df_okved[['code', 'label']].set_index('code'), 
#                           left_on='main_okved', right_index=True, how='left')\
#     .dropna()\
#     .sort_values(['id_org', 'year'])\
#     .groupby(['id_org'])['main_okved'].apply(lambda x: x.iloc[-1])

In [ ]:
df_declr_okved_amb.query('id_org == 6972')

In [ ]:
df_declr.query('id_org == 6972')['inn']

In [ ]:
# let's take the dummy approach - pick up the most earliest known main_okved value, 

## Exploration of OKVED

In [ ]:
dft = df_bld[['id_org', 'dd_okved_code_l1']].drop_duplicates()

In [ ]:
def color_red_high_values(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for large
    values, black otherwise.
    """
    color = 'red' if val >= 10 else 'black'
    return('color: %s' % color)

dft = df_bld[['id_org', 'dd_okved_code_l1']].drop_duplicates()
dft.groupby('dd_okved_code_l1').apply(lambda x: round(100*len(x)/len(dft), 1)).to_frame()\
    .rename(columns={0: 'pcts'})\
    .merge(db.df_okved2[db.df_okved2['code'].str.len() == 2][['code', 'label']].sort_values('code').set_index('code'),
           left_index=True, right_index=True, how='left')\
    .sort_index(ascending=True)
#    .style.apply(lambda x: 'color: red' if x >= 10 else 'color: black', subset=['pcts'], axis=)

In [ ]:
df_bld['dd_okved'].drop_duplicates().count()

In [ ]:
df_bld['dd_okved_code_l1'].drop_duplicates().count()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))

dft = df_bld.query('dd_okved_code_l1 != ""')[['id_org', 'dd_okved_code_l1']].groupby('dd_okved_code_l1').agg('count')

dft.plot(kind='bar', ax=ax)
if LANG =='ru':
    ax.set_xlabel('Код ОКВЭД2 (уровень 1)')
    ax.set_ylabel('Количество организаций')
elif LANG == 'en':
    ax.set_xlabel('OKVED (level 1)')
    ax.set_ylabel('Number of organizations')
    
#plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

ax.legend([])

fig.tight_layout()
fig.savefig('images/org_okved_l1.png', dpi=300)
fig.savefig('images/org_okved_l1.pdf')

# Evaluating Distribution of Factors per OKVED (to Excel)

Per each popular OKVED per each year, estimate the distribution of some parameters

In [ ]:
vars_list = ['total_consumption_tut', 'total_consumption_tut_per_emp', 'employees', 'guests']

In [ ]:
df_declr['total_consumption_tut_per_emp'] = df_declr['total_consumption_tut'] / df_declr['employees'].fillna(1)

In [ ]:
def est_distribution(data, round_precision=1):
    
    res = {}
    
    res['count_all']     = len(data)
    res['count_values']  = data.count()
    res['count_nulls']   = data.isnull().sum()
    res['min']      = data.min()
    res['q01']      = data.quantile(.01)
    res['q05']      = data.quantile(.05)
    res['q10']      = data.quantile(.10)
    res['median']   = data.median()
    res['mean']     = data.mean()
    res['q90']      = data.quantile(.90)
    res['q95']      = data.quantile(.95)
    res['q99']      = data.quantile(.99)
    res['max']      = data.max()
    
    # logger.debug(type(res['q10']))
    
    for k, v in res.items():
        if type(v) in(float, np.float64):
            res[k] = round(v, round_precision)
    
    return(pd.Series(res))

def get_rugplot_sns(data, xmedian, xlim=None):
    
    fig, axs = plt.subplots(1, 1, figsize=(5, .25))
    
    ax = axs
    
    sns.rugplot(data, height=1, ax=ax, linewidth=2, alpha=0.7)
    ax.vlines(x=xmedian, ymin=0, ymax=1, colors='red', linewidth=1)
    
    if xlim is None:
        pass
    else:
        if xlim == 'auto':
            xleft  = data[~np.isneginf(data)].min()
            xright = data[~np.isposinf(data)].max()
            # logger.debug('{} {}'.format(xleft, xright))
        elif type(xlim) in {list, tuple}:
            xleft  = xlim[0]
            xright = xlim[1]
            xlim = ax.set_xlim([xleft, xright])
        else:
            raise TypeError('Unknown type for xlim')
            
        if not np.isnan(xleft) and not np.isnan(xright):            
            
            plt.subplots_adjust(left=0.1, right=0.8)            

            plt.gcf().text(0.0,  0.25, '{:.1f}'.format(xleft),  fontsize='x-small')
            plt.gcf().text(0.82, 0.25, '{:.1f}'.format(xright), fontsize='x-small')
    
    ylim = ax.set_ylim([0, 1])
    
    yticks = ax.set_yticks([])
    yticklabels = ax.set_yticklabels([])
    xticks = ax.set_xticks([])
    xticklabels = ax.set_xticklabels([])
           
    ax.tick_params(axis='both', which='major', labelsize='x-small')
    ax.tick_params(axis='both', which='minor', labelsize='x-small')
   
    [item[1].set_linewidth(0.2)  for item in ax.spines.items()]
    [item[1].set_edgecolor('black')  for item in ax.spines.items()]
    [item[1].set_visible(True) for item in ax.spines.items()]
    
    #fig.set_frameon(False)
    #plt.autoscale(tight=True)
    #plt.subplots_adjust(top=1, bottom=0)
    
    #fig.tight_layout(pad=0)
           
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=300, bbox_inches='tight', pad_inches=0, transparent=False)    
    buf.seek(0)    
    imgdata = buf.getvalue()
    buf.close()
    plt.close()
    
    return(imgdata)


# the function below is not quite complete
def get_rugplot_mpl(data, xmedian, xlim=None):
    
    ax = axs
    
    with plt.style.context('ggplot', after_reset=True):
        
        fig, ax = plt.subplots(figsize=(4, 0.25))
        ax.plot(data, [0]*len(data), marker='|', markersize=200, color='black', linestyle=' ', alpha=0.5)
        
        yticks = ax.set_yticks([])
        yticklabels = ax.set_yticklabels([])
        # xticklabels = ax.set_xticklabels([])
        
        ax.tick_params(axis='both', which='major', labelsize=7)
        ax.tick_params(axis='both', which='minor', labelsize=7)
        
        # marking the median
        ax.vlines(x=xmedian, ymin=0, ymax=1, colors='red', linewidth=1)
    
        [item[1].set_linewidth(0.2)  for item in ax.spines.items()]
        [item[1].set_edgecolor('black')  for item in ax.spines.items()]
        [item[1].set_visible(True) for item in ax.spines.items()]
    
        if xlim != None:
            ax.set_xlim(xlim)

        ylim = ax.set_ylim([0, 1])
        
        buf = io.BytesIO()
        # fig.tight_layout()
        plt.savefig(buf, format='png', dpi=300, bbox_inches='tight', transparent=True)

        buf.seek(0)    
        imgdata = buf.getvalue()
        buf.close()
        plt.close()
            
    return(imgdata)


def get_rugplot(data, xmedian, xlim=None):
    
    return(get_rugplot_sns(data, xmedian, xlim))

# taking only the most popular okved's
okved_no_threshold = 100

Estimating the distributions and listing the outliers for the **whole** dataset.

In [ ]:
dfg = df_declr.groupby(['year', 'main_okved']).filter(lambda x: len(x) >= okved_no_threshold)\
    .groupby(['main_okved', 'year'])

# dictionary of data frame by var names
df_dict = {}

# estimating the distribution by main_okved, year
for var_name in vars_list:
    df_dict[var_name] = dfg[var_name].apply(lambda x: est_distribution(x)).to_frame().unstack()
    df_dict[var_name].columns = df_dict[var_name].columns.get_level_values(1)
    #df_dict[var_name]['var_name'] = var_name
    
#df_declr_dist = df_dict[vars_list[0]]

#for var_name in vars_list[1:]:
#    df_declr_dist = df_declr_dist.join(df_dict[var_name])

df_declr_dist = pd.concat(df_dict, axis=0)

# adding the rugplot

# the levels: 0 - var_name, 1 - okved, 2 - year
df_declr_dist['plot'] = \
    df_declr_dist.apply(lambda x: get_rugplot(df_declr.query('main_okved == "%s" and year == %s' % (x.name[1], x.name[2]))[x.name[0]].to_numpy(), 
                                              xmedian=x['median']),
                        axis=1)

                           
df_declr_outliers_dict = {}

# building the list of outliers
for var_name in vars_list:
    df_declr_outliers_dict[var_name] = df_declr.merge(df_declr_dist.loc[var_name], 
                                                      left_on=['main_okved', 'year'], 
                                                      right_index=True)\
        .query('%s < q01 or %s > q99' % (var_name, var_name))\
        [['year', 'main_okved', 'main_okved_label', 'id_org', 'company_name', 'actual_address', 
          var_name, 
          'q01', 'median', 'mean', 'q99']].sort_values(['id_org', 'year'])
    
    
# adding additional attributes to the distribution stats
print(df_declr_dist.shape)
df_declr_dist = df_declr_dist.merge(db.df_okved[['code', 'label']].set_index('code').rename(columns={'label': 'main_okved_label'}), 
                                    left_on='main_okved', right_index=True, how='left')
# forward the required columns
df_declr_dist = df_declr_dist.reindex(columns=pd.Index(['main_okved_label']).append( df_declr_dist.columns.difference(['main_okved_label'])))
print(df_declr_dist.shape)

Estimating the distributions and listing the outliers for a subset (within the quantiles 0.1-0.9) per each variable per each main_okved and year

In [ ]:
dfg2 = df_declr.groupby(['year', 'main_okved'])\
    .filter(lambda x: len(x) >= okved_no_threshold).groupby(['main_okved', 'year'])

# dictionary of data frame by var names
df_dict2 = {}

# estimating the distribution by main_okved, year
for var_name in vars_list:
    df_dict2[var_name] = dfg2[var_name]\
        .apply(lambda x: est_distribution(x[(x >= df_declr_dist.loc[(var_name, x.name[0], x.name[1])]['q10']) & \
                                            (x <= df_declr_dist.loc[(var_name, x.name[0], x.name[1])]['q90'])])).to_frame().unstack()
    df_dict2[var_name].columns = df_dict2[var_name].columns.get_level_values(1)
    #df_dict[var_name]['var_name'] = var_name
    
#df_declr_dist = df_dict[vars_list[0]]

#for var_name in vars_list[1:]:
#    df_declr_dist = df_declr_dist.join(df_dict[var_name])

df_declr_dist2 = pd.concat(df_dict2, axis=0)

# adding the rugplot

def invoke_get_rugplot(x):
    
    (var_name, main_okved, year) = x.name
    ql = df_declr_dist.loc[x.name]['q10']
    qh = df_declr_dist.loc[x.name]['q90']
    median = x['median']
    xleft  = df_declr_dist2.loc[(var_name, main_okved)]['min'].replace([np.inf, -np.inf], np.nan).min()
    xright = df_declr_dist2.loc[(var_name, main_okved)]['max'].replace([np.inf, -np.inf], np.nan).max()
    
    # logger.debug('{} {} {} {:.1f} {:.1f}'.format(var_name, main_okved, year, xleft, xright))
    
    x = df_declr.query('main_okved == "{main_okved}" and year == {year} and {var_name} >= {ql} and {var_name} <= {qh}'\
                                    .format(var_name=var_name,
                                            main_okved=main_okved, 
                                            year=year, 
                                            ql=ql,
                                            qh=qh))[var_name].to_numpy()
    
    imgdata = get_rugplot(x, xmedian=median, xlim=(xleft, xright))

    return(imgdata)

df_declr_dist2['plot'] = df_declr_dist2.apply(lambda x: invoke_get_rugplot(x), axis=1)


df_declr_outliers_dict2 = {}

# building the list of outliers
for var_name in vars_list:
    df_declr_outliers_dict2[var_name] = df_declr.merge(df_declr_dist2.loc[var_name], 
                                                       left_on=['main_okved', 'year'], 
                                                       right_index=True)\
        .query('%s < q10 or %s > q90' % (var_name, var_name))\
        [['year', 'main_okved', 'main_okved_label','id_org', 'company_name', 'actual_address', 
          var_name, 
          'q10', 'median', 'mean', 'q90']].sort_values(['id_org', 'year'])
    
    
# adding additional attributes to the distribution stats
print(df_declr_dist2.shape)
df_declr_dist2 = df_declr_dist2.merge(db.df_okved[['code', 'label']].set_index('code').rename(columns={'label': 'main_okved_label'}), 
                                     left_on='main_okved', right_index=True, how='left')
# forward the required columns
df_declr_dist2 = df_declr_dist2.reindex(columns=pd.Index(['main_okved_label']).append( df_declr_dist2.columns.difference(['main_okved_label'])))
print(df_declr_dist2.shape)      

In [ ]:
# data = df_declr.query('main_okved == "{main_okved}" and year == {year} and {var_name} >= {ql} and {var_name} <= {qh}'\
#           .format(main_okved="75.11.31", 
#                   year=2014, 
#                   var_name='employees',
#                   ql=0.5,
#                   qh=20.5))['employees'].to_numpy()

# from PIL import Image

# imgdata = get_rugplot(data, xmedian=10, xlim=(6, 30))

# im = Image.open(io.BytesIO(imgdata))
# im

In [ ]:
# an example of image retrieval

# from PIL import Image
# buf = io.BytesIO(df_declr_dist2.loc[('employees', '75.11.31', 2014)]['plot'])
# im = Image.open(buf)
# im

## Export

In [ ]:
import string

exports = {'All': 
             {
             'file_name': './export/{}'.format('activity_factors_distr_by_okved_all.xlsx'),
             'sheets':   {
                  '1. Оц. распр. tct':          {'data': df_declr_dist.loc['total_consumption_tut'].reset_index()},
                  '1. Оц. распр. tct_per_emp':  {'data': df_declr_dist.loc['total_consumption_tut_per_emp'].reset_index()},
                  '1. Оц. распр. employees':    {'data': df_declr_dist.loc['employees'].reset_index()},
                  '1. Оц. распр. guests':       {'data': df_declr_dist.loc['guests'].reset_index()},          
                  '1. Выбросы по ttt':          {'data': df_declr_outliers_dict['total_consumption_tut']},
                  '1. Выбросы по ttt_per_emp':  {'data': df_declr_outliers_dict['total_consumption_tut_per_emp']},
                  '1. Выбросы по employees':    {'data': df_declr_outliers_dict['employees']},
                  '1. Выбросы по guests':       {'data': df_declr_outliers_dict['guests']},
                 }
             },
          'Subset1':
             {
             'file_name': './export/{}'.format('activity_factors_distr_by_okved_without_outliers.xlsx'),
             'sheets':   {
                  '2. Оц. распр. tct':          {'data': df_declr_dist2.loc['total_consumption_tut'].reset_index()},
                  '2. Оц. распр. tct_per_emp':  {'data': df_declr_dist2.loc['total_consumption_tut_per_emp'].reset_index()},
                  '2. Оц. распр. employees':    {'data': df_declr_dist2.loc['employees'].reset_index()},
                  '2. Оц. распр. guests':       {'data': df_declr_dist2.loc['guests'].reset_index()}
                 }
             }
         }
                 
for export_type, export_config in exports.items():

    file_name = export_config['file_name']

    xls_writer = pd.ExcelWriter(file_name,
                                engine='xlsxwriter',
                                options={'strings_to_urls': False})  # 'strings_to_numbers': True

    logger.info('Saving to %s' % file_name)

    for key, val in export_config['sheets'].items():

        # Convert the dataframe to an XlsxWriter Excel object.
        sheet_name = key

        logger.debug('The sheet name: %s' % key)

        dft = val['data']

        rename_map = {'year':           'Год', 
                      'main_okved':     'ОКВЭД', 
                      'main_okved_label': 'Расщифровка ОКВЭД', 
                      'company_name':   'Имя организации',
                      'actual_address': 'Адрес организации (факт.)',
                      'count_all':      'Кол-во всех записей',
                      'count_values':   'Кол-во заполненных',
                      'count_nulls':    'Кол-во пустых записей',
                      'guests': 'Посетителей',
                      'min':    'Минимум',
                      'q01':    '0.01-квантиль',
                      'q05':    '0.05-квантиль',
                      'q10':    '0.10-квантиль',
                      'median': 'медиана',
                      'mean':   'арифм. среднее',
                      'q90':    '0.90-квантиль',
                      'q95':    '0.95-квантиль',
                      'q99':    '0.99-квантиль',
                      'max':    'Максимум',
                      'plot':   'plot'}

        # enrich the renaming map with the missing values
        missing_col_names = {col_name: col_name for col_name in dft.columns.difference(rename_map)}
        if len(missing_col_names) > 0:
            logger.debug('Expanding the renaming map with %s' % missing_col_names)
            rename_map.update(missing_col_names)

        dft.columns = dft.columns.map(rename_map)

        dft.loc[:, dft.columns != 'plot'].to_excel(xls_writer, index=False, sheet_name=sheet_name)

        # Set the column width and format.
        workbook  = xls_writer.book
        worksheet = xls_writer.sheets[sheet_name]
        sheet_data_range = (string.ascii_uppercase[0] + '1', 
                            string.ascii_uppercase[val['data'].shape[1]] + str(val['data'].shape[0] + 1))
        worksheet.set_column('A:Z', width=15,  cell_format=workbook.add_format().set_align('left'))

        # https://github.com/jmcnamara/XlsxWriter/issues/111
        if 'Оц. ' in key:
            cell_format1 = workbook.add_format()
            cell_format1.set_num_format('0.00')
            cell_format1.set_align('left')
            worksheet.set_column('A:Z', width=10, cell_format=cell_format1)
            worksheet.set_column('B:E', width=10, cell_format=workbook.add_format({'num_format': '0'}))
            # rotation of labels
            worksheet.set_column('C1:Q1', cell_format=workbook.add_format().set_rotation(90))

        #worksheet.set_column('B:F', width = 10,  cell_format = workbook.add_format().set_align('center'))
        #worksheet.set_column('G:G', width = 100, cell_format = workbook.add_format().set_align('left'))

        # if there is the 'plot' column, insert its values to the left of table
        if 'plot' in set(dft.columns):
            row = 2
            for i, v in dft.iterrows():
                cell_addr = 'Q%d' % row
                # logger.debug('Inserting image into %s' % cell_addr)
                worksheet.insert_image(cell_addr, 'rugplot_%s_%d.png' % (v['ОКВЭД'], v['Год']),
                                       {'image_data': io.BytesIO(v['plot'])})
                row += 1

        # Set the autofilter.
        worksheet.autofilter('%s:%s' % sheet_data_range)
        worksheet.freeze_panes(1, 2)

    xls_writer.save()

# Employees

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld.groupby('year').agg({'employees': ['mean', 'median']})
dft.columns = pd.Index(['mean', 'median'])
dft.plot(kind='bar', ax=ax, alpha=0.9)

ax.legend(loc = 'lower right', frameon=True)

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Сотрудников, чел.')
elif LANG == 'en':     
    ax.set_xlabel('Year')
    ax.set_ylabel('Employees')
    
ax.tick_params(axis='x', labelrotation=0)
    
fig.tight_layout()
fig.savefig('images/bld_emp.png', dpi=300)
fig.savefig('images/bld_emp.pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

#dft = df_bld.query('heated_area <= 100000')
dft = df_bld[['year', 'employees']].query('employees > 0')
dft['employees_log10'] = np.log10(dft['employees'])

#sns.catplot(x='year', y='heated_area_log10', data=dft, kind='violin') #, width=0.6)
sns.boxplot(x='year', y='employees_log10', data=dft, width=0.6, color='#33CC66')

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Сотрудников, чел.')
elif LANG == 'en':
    ax.set_xlabel('Year')
    ax.set_ylabel('$log_{10}{employees}$')
    
#plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)

ax.set_yticks(range(4))
#ax.set_yticklabels([str(n) for n in range(4)])

fig.tight_layout()
fig.savefig('images/bld_emp_boxplot.png', dpi=300)
fig.savefig('images/bld_emp_boxplot.pdf')

# Guests

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld.groupby('year').agg({'guests': ['mean', 'median']})
dft.columns = pd.Index(['mean', 'median'])
dft.plot(kind='bar', ax=ax, alpha=0.9)

ax.legend(loc = 'lower right', frameon=True)

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Посетителей, чел.')
elif LANG == 'en':     
    ax.set_xlabel('Year')
    ax.set_ylabel('Visitors')
    
ax.tick_params(axis='x', labelrotation=0)
    
fig.tight_layout()
fig.savefig('images/bld_guests.png', dpi=300)
fig.savefig('images/bld_guests.pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

#dft = df_bld.query('heated_area <= 100000')
dft = df_bld[['year', 'guests']].query('guests > 0')
dft['guests_log10'] = np.log10(dft['guests'])

#sns.catplot(x='year', y='heated_area_log10', data=dft, kind='violin') #, width=0.6)
sns.boxplot(x='year', y='guests_log10', data=dft, width=0.6, color='#33CC66')

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Посетителей, чел.')
elif LANG == 'en':
    ax.set_xlabel('Year')
    ax.set_ylabel('$log_{10}{visitors}$')
    
ax.set_yticks(range(8))
#ax.set_yticklabels([str(n) for n in range(4)])

fig.tight_layout()
fig.savefig('images/bld_guests_boxplot.png', dpi=300)
fig.savefig('images/bld_guests_boxplot.pdf')

# Some Attributes Over Years

## Procedures

In [ ]:
def plot_distr(fig, axs, data, field_list, quantile_thr = 1, title=None, median=False):

    for i, f in enumerate(field_list):
        x = data[~data[f].isnull() & (data[f] > 0) & (data[f] < data[f].quantile(quantile_thr))][f]
        ax = axs[0][i]
        p = sns.distplot(x, ax=ax)
        if median:
            ax.axvline(x.median(), color='r')
        # ax.annotate('median=%.2f' % x.median(), (0, 0.9))
        p = sns.boxplot(x, ax=axs[1][i])

    t = fig.suptitle(title, size=15)

In [ ]:
def plot_distr_over_years(fig, ax, df, ylabel='Потребление ресурсов'):

    '''
    df is expected to be some value with the year in the index
    '''
    dft = df

    dft.plot(kind='bar', ax=ax)
    ax.set_xlabel('Год')
    ax.set_ylabel(ylabel)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)

    fig.tight_layout()

## Energy Consumption Over Years

In [ ]:
df_ee['electric_unit'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_ee.merge(df_bld, left_on='id_declr_building', right_index=True)[['year', 'electric_consumption']]
dft = dft.groupby('year')['electric_consumption'].sum()/1000000

dft.plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Электропотребление, млн. кВт*ч')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig('images/ee_years.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_ee.merge(df_bld, left_on='id_declr_building', right_index=True)[['year', 'electric_payment']]
dft = dft.groupby('year')['electric_payment'].sum()/1000000

dft.plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Электропотребление, млн. руб')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig('images/ee_payment_years.png', dpi=300)

In [ ]:
df_cw['cold_water_unit'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_cw.merge(df_bld, left_on='id_declr_building', right_index=True)[['year', 'cold_water_consumption']]
dft = dft.groupby('year')['cold_water_consumption'].sum()/1000000

dft.plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Холодной воды, млн. куб. м')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig('images/cw_years.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_cw.merge(df_bld, left_on='id_declr_building', right_index=True)[['year', 'cold_water_payment']]
dft = dft.groupby('year')['cold_water_payment'].sum()/1000000

dft.plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Холодной воды, млн. руб.')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig('images/cw_payment_years.png', dpi=300)

In [ ]:
df_hw['hot_water_unit'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_hw.merge(df_bld, left_on='id_declr_building', right_index=True)[['year', 'hot_water_consumption']]
dft = dft.groupby('year')['hot_water_consumption'].sum()/1000000

dft.plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Горячей воды, млн. куб. м')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig('images/hw_years.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_hw.merge(df_bld, left_on='id_declr_building', right_index=True)[['year', 'hot_water_payment']]
dft = dft.groupby('year')['hot_water_payment'].sum()/1000000

dft.plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Горячей воды, млн. руб.')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig('images/hw_payment_years.png', dpi=300)

## Heat Consumption Over Years

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld[['year', 'heat_consumption']].groupby('year')['heat_consumption'].sum()/1000000

dft.plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Потребление тепла, млн. Гкал')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig('images/heat_years.png', dpi=300)

## Distribution of Certain Parameters (Over Years)

In [ ]:
# guests for 80.21 "Основное общее и среднее (полное) общее образование"

def q05(x):
    return(np.quantile(x, .05))

def q95(x):
    return(np.quantile(x, .95))

df_declr[df_declr['main_okved'].fillna('').str.startswith('80.10.2')][['year', 'guests']]\
    .pivot_table(columns='year', aggfunc={'count', min, q05, np.median, np.mean, max, q95}).round(1)

In [ ]:
for param_name in ['guests', 'employees', 'building_amount', 'id_org',
                   'total_consumption_tut',
                   'electric_per_floor_area', 'electric_per_useful_area', 'electric_per_employees_and_guests',
                   'cold_water_per_useful_area', 'cold_water_per_employees_and_guests',
                   'hot_water_per_employees_and_guests',
                   'heat_per_heated_area', 'heat_per_employees_and_guests']:
                   #'gas_per_heated_area', 'gas_per_employees_and_guests'
                   #'tut_per_heated_area', 'tut_per_floor_area', 'tut_per_employees_and_guests']:

    fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(16,6), gridspec_kw = {'height_ratios':[7, 1]})

    for i, y in enumerate([2014, 2015, 2016, 2017]):

        data = df_declr.query('year == %d' % y)

        for j, f in enumerate([param_name]):
            
            if f in {'id_org'}:
                quantile_thr_lo = 0
                quantile_thr_hi = 1
            else:
                quantile_thr_lo = 0.05
                quantile_thr_hi = 0.95
                
            x = data[~data[f].isnull() & 
                     (data[f] > 0) & 
                     (data[f] >= data[f].quantile(quantile_thr_lo)) &
                     (data[f] <= data[f].quantile(quantile_thr_hi))][f]
            
            # distribution plot
            ax_dp = axs[j][i]
            p = sns.distplot(x, ax=ax_dp)
            ax_dp.set_title('Случаев: {}'.format(len(x)))
            ax_dp.set_xlabel('')
            if True:
                ax_dp.axvline(x.median(), color='r')
                ax_dp.annotate('median=%.2f' % x.median(), (0.5, 0.9), xycoords='axes fraction')
                
            # box plot
            ax_bp = axs[j+1][i]            
            p = sns.boxplot(x, ax=ax_bp)
            ax_bp.set_xlabel('')
            ax_bp.set_xlim(ax_dp.get_xlim())

            #t = ax.set_title('%d' % y)

    t = fig.suptitle('{} по годам ({:.2f}-{:.2f}-квантили)'.format(param_name, quantile_thr_lo, quantile_thr_hi), size=15)

    # fig.tight_layout()
    fig.savefig('images/{}.png'.format(param_name), dpi=300)

## total_consumption_tut - testing whether they are different distributions

In [ ]:
from scipy.stats import ks_2samp

year_list = (2014, 2015, 2016, 2017)

var_name = 'total_consumption_tut'
print(var_name)
for y1 in year_list:
    for y2 in year_list:
        if y1 < y2:
            print('%d vs %d: ' % (y1, y2), end='')
            print(ks_2samp(df_declr.query('year == %d' % y1)[var_name], df_declr.query('year == %d' % y2)[var_name]))

Looks like 2017 year is different from the previous years.

# Energy Efficiency Rating

In [ ]:
df_bld.groupby('dd_okved')['dd_okved'].count().sort_values(ascending=False).head(5)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 7))

plt.subplots_adjust(wspace=1)

year=2017
dd_okved='85.13'
#main_okved='all'

if main_okved == 'all':
    dft = df_bld_tmp.query('year==@year')
else:
    dft = df_bld_tmp\
        .merge(df_bld['dd_okved'], left_on='id', right_index=True, how='left')\
        .query('year==@year and dd_okved==@dd_okved')

dft['electric_per_floor_area_log']           = np.log10(dft['electric_per_floor_area'])
dft['electric_per_employees_and_guests_log'] = np.log10(dft['electric_per_employees_and_guests'])
   
ax = axs[0]
p = sns.scatterplot(data=dft, 
                    x='electric_points', 
                    y='electric_per_floor_area_log', 
                    hue='efficiency_class_name', palette='RdYlGn_r', hue_order=['A','B','C','D','E','F','G'],
                    linewidth=0, alpha=0.8, ax=ax)

if LANG == 'en':    
    ax.set_xlabel('electric points')
    ax.set_ylabel('electric consumption per floor area, $log_{10}$ scale')

ax = axs[1]
p = sns.scatterplot(data=dft, 
                    x='electric_points', 
                    y='electric_per_employees_and_guests_log', 
                    hue='efficiency_class_name', palette='RdYlGn_r', hue_order=['A','B','C','D','E','F','G'],
                    linewidth=0, alpha=0.8, ax=ax)

if LANG == 'en':    
    ax.set_xlabel('electric points')
    ax.set_ylabel('electric consumption per persons, $log_{10}$ scale')

#ax.set_yscale('log')

# suptitle = 'Распределение класса энергоэффективности здания в зависимости от\n'\
#            'баллов (по электроэнергии) и удельных показателей\n'\
#            '(данные за {} год) для'.format(year)

# if main_okved == 'all':
#     suptitle = suptitle + ' всех зданий'
# else:
#     suptitle = suptitle + ' зданий с основным ОКВЭД {}'.format(main_okved)

# fig.suptitle(suptitle)

fig.tight_layout()
fig.subplots_adjust(top=0.85)

fig.savefig(f'images/ec_ee_efficiency_class_{year}_{dd_okved}.jpg', dpi=300)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 7))

plt.subplots_adjust(wspace=1)

year=2017
dd_okved='85.13'
#main_okved='all'

if main_okved == 'all':
    dft = df_bld_tmp.query('year==@year')
else:
    dft = df_bld_tmp\
        .merge(df_bld['dd_okved'], left_on='id', right_index=True, how='left')\
        .query('year==@year and dd_okved==@dd_okved')

dft['heat_per_heated_area_log']           = np.log10(dft['heat_per_heated_area'])
dft['heat_per_employees_and_guests_log']  = np.log10(dft['heat_per_employees_and_guests'])
   
ax = axs[0]
p = sns.scatterplot(data=dft, 
                    x='heat_points', 
                    y='heat_per_heated_area_log', 
                    hue='efficiency_class_name', palette='RdYlGn_r', hue_order=['A','B','C','D','E','F','G'],
                    linewidth=0, alpha=0.8, ax=ax)

if LANG == 'en':    
    ax.set_xlabel('heat points')
    ax.set_ylabel('heat consumption per heated area, $log_{10}$ scale')

ax = axs[1]
p = sns.scatterplot(data=dft, 
                    x='heat_points', 
                    y='heat_per_employees_and_guests_log', 
                    hue='efficiency_class_name', palette='RdYlGn_r', hue_order=['A','B','C','D','E','F','G'],
                    linewidth=0, alpha=0.8, ax=ax)

if LANG == 'en':    
    ax.set_xlabel('heat points')
    ax.set_ylabel('heat consumption per persons, $log_{10}$ scale')

#ax.set_yscale('log')

# suptitle = 'Распределение класса энергоэффективности здания в зависимости от\n'\
#            'баллов (по электроэнергии) и удельных показателей\n'\
#            '(данные за {} год) для'.format(year)

# if main_okved == 'all':
#     suptitle = suptitle + ' всех зданий'
# else:
#     suptitle = suptitle + ' зданий с основным ОКВЭД {}'.format(main_okved)

# fig.suptitle(suptitle)

fig.tight_layout()
fig.subplots_adjust(top=0.85)

fig.savefig(f'images/ec_ht_efficiency_class_{year}_{dd_okved}.jpg', dpi=300)

# Checks for 'Fraud'

In [ ]:
def chisq_stat(O, E):
    return(sum( [(o - e)**2/e for (o, e) in zip(O, E)] ))

## Employees

In [ ]:
dft = df_bld['employees'].dropna().astype('str').str.slice(0, 1).astype('int').rename('observed')
benford_observed = dft[dft != 0].value_counts().sort_index()

benford_expected = pd.Series([len(dft)*np.log10(1 + 1./i) for i in range(1, 10)], index=range(1, 10), name='expected')
benford = pd.concat([benford_observed, benford_expected], axis=1)

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
ax = axs
benford.plot(kind='bar', ax=ax)
ax.legend(['observed', 'expected'])

ax.set_xlabel('First digit')
ax.set_ylabel('Frequency')

ax.tick_params(axis='x', labelrotation=0)

fig.tight_layout()
fig.savefig('images/bld_employees_benford.png', dpi=300)
fig.savefig('images/bld_employees_benford.pdf')

print(chisq_stat(benford['observed'].to_numpy(), benford['expected'].to_numpy()))

## Guests

In [ ]:
dft = df_bld['guests'].dropna().astype('str').str.slice(0, 1).astype('int').rename('observed')
benford_observed = dft[dft != 0].value_counts().sort_index()

benford_expected = pd.Series([len(dft)*np.log10(1 + 1./i) for i in range(1, 10)], index=range(1, 10), name='expected')
benford = pd.concat([benford_observed, benford_expected], axis=1)

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
ax = axs
benford.plot(kind='bar', ax=ax)
ax.legend(['observed', 'expected'])

ax.set_xlabel('First digit')
ax.set_ylabel('Frequency')

ax.tick_params(axis='x', labelrotation=0)

fig.tight_layout()

fig.savefig('images/bld_guests_benford.png', dpi=300)
fig.savefig('images/bld_guests_benford.pdf')

print(chisq_stat(benford['observed'].to_numpy(), benford['expected'].to_numpy()))

# Heating

In [ ]:
dft = df_bld['heat_consumption'].dropna().astype('str').str.slice(0, 1).astype('int').rename('observed')
benford_observed = dft[dft != 0].value_counts().sort_index()

benford_expected = pd.Series([len(dft)*np.log10(1 + 1./i) for i in range(1, 10)], index=range(1, 10), name='expected')
benford = pd.concat([benford_observed, benford_expected], axis=1)

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
ax = axs
benford.plot(kind='bar', ax=ax)
ax.legend(['observed', 'expected'])

ax.tick_params(axis='x', labelrotation=0)

fig.tight_layout()

fig.savefig('images/bld_ht_consumption_benford.png', dpi=300)
fig.savefig('images/bld_ht_consumption_benford.pdf')

print(chisq_stat(benford['observed'].to_numpy(), benford['expected'].to_numpy()))

# Recommendations

## On Data Input Validation

1. Есть пропущенные данные.
2. Некоторые организации указывали разные ОКВЭД для разных годов.
3. Количество сотрудников иногда задано некорректно?
4. Количество гостей задаётся оценочно.
5. Есть здания, по которым нулевое потребление электроэнергии.